In [1]:
HOST = "10.13.128.11"
USER = "postgres"
PW="asfdfdsa"
DB="postgres"

In [2]:
import psycopg

def connect():
    
    url = "dbname='{}' user='{}' host={} port=5432 password={}".format(DB, USER, HOST, PW)
    connection = psycopg.connect(url)

    return connection

In [3]:
import psycopg

connection = connect()

print(connection)

<psycopg.Connection [IDLE] (host=10.13.128.11 database=postgres) at 0x7fb342927580>


In [32]:
import psycopg

connection = connect()
print(connection)

drop = 'DROP TABLE IF EXISTS final_project.reservations'

create_schema = 'CREATE SCHEMA IF NOT EXISTS final_project'

create = '''CREATE TABLE final_project.reservations (
    res_id SERIAL PRIMARY KEY,
    cust_id INTEGER NOT NULL,
    prp_nm VARCHAR(25) NOT NULL,
    prp_ch VARCHAR(25) NOT NULL,
    adr_line_1 VARCHAR(50) NOT NULL,
    adr_line_2 VARCHAR(50) NOT NULL,
    city VARCHAR(30) NOT NULL, 
    state CHAR(2) NOT NULL, 
    postal_cd CHAR(5) NOT NULL,
    lat NUMERIC(9, 6),
    long NUMERIC(9, 6), 
    cnt_code VARCHAR(3) NOT NULL,
    arr_date DATE NOT NULL,
    dep_date DATE NOT NULL,
    pmt_amt DECIMAL(10, 2) NOT NULL 
) '''

print(drop)
print(create)

try:
    
    cursor = connection.cursor()
    cursor.execute(drop)
    print('Dropped reservations table')
    
    cursor.execute(create_schema)
    print('Created schema final_project')
    
    cursor = connection.cursor()
    cursor.execute(create)
    print('Created reservations table')
    
    connection.commit() # commit the changes 
except Exception as e:
    print("Error occurred while creating reservations table:", e)

<psycopg.Connection [IDLE] (host=10.13.128.11 database=postgres) at 0x7fb3404e3730>
DROP TABLE IF EXISTS final_project.reservations
CREATE TABLE final_project.reservations (
    res_id SERIAL PRIMARY KEY,
    cust_id INTEGER NOT NULL,
    prp_nm VARCHAR(25) NOT NULL,
    prp_ch VARCHAR(25) NOT NULL,
    adr_line_1 VARCHAR(50) NOT NULL,
    adr_line_2 VARCHAR(50) NOT NULL,
    city VARCHAR(30) NOT NULL, 
    state CHAR(2) NOT NULL, 
    postal_cd CHAR(5) NOT NULL,
    lat NUMERIC(9, 6),
    long NUMERIC(9, 6), 
    cnt_code VARCHAR(3) NOT NULL,
    arr_date DATE NOT NULL,
    dep_date DATE NOT NULL,
    pmt_amt DECIMAL(10, 2) NOT NULL 
) 
Dropped reservations table
Created schema final_project
Created reservations table


In [33]:
import psycopg
from faker import Faker
import random

connection = connect()
# print(connection)

Faker.seed(1234)

fake = Faker()

possible_cust_ids = list(range(1, 51))

sql = "INSERT INTO final_project.reservations (cust_id, prp_nm, prp_ch, adr_line_1, adr_line_2, city, state, postal_cd, lat, long, cnt_code, arr_date, dep_date, pmt_amt) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    cursor = connection.cursor()

    for _ in range(100):
        cust_id = fake.random_element(possible_cust_ids)
        
        rec = (
            cust_id, 
            fake.word(),   
            fake.word(),  
            fake.street_address(),  
            fake.secondary_address(),  
            fake.city(),   
            fake.state_abbr(), 
            fake.zipcode(), 
            fake.latitude(),  
            fake.longitude(), 
            fake.country_code(),  
            fake.date_this_decade(),  
            fake.date_this_decade(),  
            fake.pydecimal(left_digits=5, right_digits=2)  
        )

        cursor.execute(sql, rec)

    connection.commit()
    print("100 records written into reservations table")

except Exception as e:
    print("Failed to insert records into reservations table:", e)

100 records written into reservations table


In [25]:
!psql -c "\d final_project.reservations"

                                           Table "final_project.reservations"
   Column   |         Type          | Collation | Nullable |                          Default                           
------------+-----------------------+-----------+----------+------------------------------------------------------------
 res_id     | integer               |           | not null | nextval('final_project.reservations_res_id_seq'::regclass)
 cust_id    | integer               |           | not null | 
 prp_nm     | character varying(25) |           | not null | 
 prp_ch     | character varying(25) |           | not null | 
 adr_line_1 | character varying(50) |           | not null | 
 adr_line_2 | character varying(50) |           | not null | 
 city       | character varying(30) |           | not null | 
 state      | character(2)          |           | not null | 
 postal_cd  | character(5)          |           | not null | 
 lat        | numeric(9,6)          |           |          | 
 l

In [27]:
!psql -c "select * from final_project.reservations"

 res_id | cust_id |   prp_nm    |   prp_ch    |            adr_line_1            | adr_line_2 |        city         | state | postal_cd |    lat     |    long     | cnt_code |  arr_date  |  dep_date  |  pmt_amt  
--------+---------+-------------+-------------+----------------------------------+------------+---------------------+-------+-----------+------------+-------------+----------+------------+------------+-----------
      1 |      50 | let         | but         | 90115 Andre Turnpike             | Apt. 597   | Hernandezburgh      | VT    | 15497     | -86.075802 |   90.090998 | RU       | 2023-06-13 | 2020-12-23 | -19189.04
      2 |      36 | at          | quite       | 5433 Mcdowell Shores Apt. 810    | Suite 276  | North Charles       | NM    | 21997     | -69.780986 |  119.214393 | GM       | 2020-07-25 | 2023-09-02 |  82470.64
      3 |      43 | sure        | amount      | 2427 Anna Track                  | Suite 759  | Alvarezfort         | VI    | 78477     | -44.373662 |